In [1]:
from satsearch import Search
import stackstac, os, requests
from netrc import netrc
from subprocess import Popen
from getpass import getpass
import rasterio
from distributed import LocalCluster,Client

### Configuration

In [33]:
data = 'hls'
#data = 'sentinel_2a'
s3 = False

In [8]:
if data == 'hls':
    #Setup NASA Credentials
    urs = 'urs.earthdata.nasa.gov'    # Earthdata URL to call for authentication
    prompts = ['Enter NASA Earthdata Login Username \n(or create an account at urs.earthdata.nasa.gov): ',
               'Enter NASA Earthdata Login Password: ']
    try:
        netrcDir = os.path.expanduser("~/.netrc")
        netrc(netrcDir).authenticators(urs)[0]
        del netrcDir

    # Below, create a netrc file and prompt user for NASA Earthdata Login Username and Password
    except FileNotFoundError:
        homeDir = os.path.expanduser("~")
        Popen('touch {0}.netrc | chmod og-rw {0}.netrc | echo machine {1} >> {0}.netrc'.format(homeDir + os.sep, urs), shell=True)
        Popen('echo login {} >> {}.netrc'.format(getpass(prompt=prompts[0]), homeDir + os.sep), shell=True)
        Popen('echo password {} >> {}.netrc'.format(getpass(prompt=prompts[1]), homeDir + os.sep), shell=True)
        del homeDir, urs, prompts

In [9]:
if s3:
    #Get NASA Temp AWS Credentials
    s3_cred = requests.get('https://lpdaac.earthdata.nasa.gov/s3credentials').json()
    s3_cred
    
    env = dict(GDAL_DISABLE_READDIR_ON_OPEN='YES', 
           GDAL_MAX_RAW_BLOCK_CACHE_SIZE='200000000',
           GDAL_SWATH_SIZE='200000000',
           VSI_CURL_CACHE_SIZE='200000000',
           CPL_VSIL_CURL_ALLOWED_EXTENSIONS='TIF',
           GDAL_HTTP_UNSAFESSL='YES',
           GDAL_HTTP_COOKIEFILE=os.path.expanduser('~/cookies.txt'),
           GDAL_HTTP_COOKIEJAR=os.path.expanduser('~/cookies.txt'),
           AWS_REGION='us-west-2',
           AWS_SECRET_ACCESS_KEY=s3_cred['secretAccessKey'],
           AWS_ACCESS_KEY_ID=s3_cred['accessKeyId'],
           AWS_SESSION_TOKEN=s3_cred['sessionToken'])
    os.environ.update(env)

# Default to the StackStac ~.LayeredEnv
# else:
#     env = dict(GDAL_DISABLE_READDIR_ON_OPEN='EMPTY_DIR', 
#                    AWS_NO_SIGN_REQUEST='YES',
#                    GDAL_MAX_RAW_BLOCK_CACHE_SIZE='200000000',
#                    GDAL_SWATH_SIZE='200000000',
#                    VSI_CURL_CACHE_SIZE='200000000',
#                    GDAL_HTTP_COOKIEFILE=os.path.expanduser('~/cookies.txt'),
#                    GDAL_HTTP_COOKIEJAR=os.path.expanduser('~/cookies.txt'))
#     os.environ.update(env)

In [35]:
def get_STAC_items(url, collection, dates, bbox):
  results = Search.search(url=url,
                      collections=collection, 
                      datetime=dates,
                      bbox=bbox)

  return(results)

def remap_s3_url(stac):
    for i,entry in enumerate(stac):
        for asset in entry['assets'].keys():
            stac[i]['assets'][asset]['href'] = stac[i]['assets'][asset]['href'].replace('https://lpdaac.earthdata.nasa.gov/',
                                                                                          '/vsis3/')
    return(stac)
if data == 'hls':
    url = 'https://cmr.earthdata.nasa.gov/stac/LPCLOUD/' 
    collection = ['HLSS30.v1.5']#'C1711924822-LPCLOUD' #HLS
    bbox=[-104.79107047,   40.78311181, -104.67687336,   40.87008987]
    dates = '2020-01-01/2021-02-10'

    stac_items = get_STAC_items(url,collection,dates,','.join(map(str, bbox))).items()
    s_col = stac_items.geojson()['features']
    if s3:
        s_col = remap_s3_url(s_col)
if data == 'sentinel_2a':
    collection = ['sentinel-s2-l2a-cogs']
    bbox=[-104.79107047,   40.78311181, -104.67687336,   40.87008987]
    dates = '2013-01-01/2022-02-10'

    stac_items = get_STAC_items(url,collection,dates,bbox).items()
    s_col = stac_items.geojson()['features']
    if s3:
        s_col = remap_s3_url(s_col)
print('Number of Items: ',len(s_col))

Number of Items:  45


In [30]:
with rasterio.open(s_col[0]['assets']['B01']['href']) as src:
    b= src.bounds
    prof = src.profile
    dtype = prof['dtype']
    nodata = prof['nodata']
    print(prof)

{'driver': 'GTiff', 'dtype': 'uint16', 'nodata': 0.0, 'width': 1830, 'height': 1830, 'count': 1, 'crs': CRS.from_epsg(32613), 'transform': Affine(60.0, 0.0, 499980.0,
       0.0, -60.0, 4600020.0), 'blockxsize': 256, 'blockysize': 256, 'tiled': True, 'compress': 'deflate', 'interleave': 'band'}


In [6]:
#lab_extension = /user/<username>/proxy/8787/status
cluster = LocalCluster(nthreads=2)
cl = Client(cluster)
cl

Client Scheduler: tcp://127.0.0.1:46859 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 8.59 GB


In [8]:
da = stackstac.stack(s_col,dtype='int16',fill_value=-9999.,resolution=30,epsg=32613,chunksize=36000,assets=['B01'])#,bounds=list(b))#,gdal_env=stackstac.LayeredEnv(env))
da

<xarray.DataArray 'stackstac-3124560797a5d8d018939656d982a643' (time: 12, band: 1, y: 3715, x: 3717)>
dask.array<fetch_raster_window, shape=(12, 1, 3715, 3717), dtype=int16, chunksize=(1, 1, 3715, 3717), chunktype=numpy.ndarray>
Coordinates:
  * time            (time) object 1609524157093000000 ... 1612720360118000000
    id              (time) <U19 'G1990200426-LPCLOUD' ... 'G2003169005-LPCLOUD'
  * band            (band) <U3 'B01'
  * x               (x) float64 5e+05 5e+05 5e+05 ... 6.114e+05 6.114e+05
  * y               (y) float64 4.601e+06 4.601e+06 ... 4.489e+06 4.489e+06
    start_datetime  (time) <U24 '2021-01-01T18:02:37.093Z' ... '2021-02-07T17...
    eo:cloud_cover  (time) int64 56 16 14 8 30 79 44 98 67 0 42 39
    end_datetime    (time) <U24 '2021-01-01T18:02:37.093Z' ... '2021-02-07T17...
Attributes:
    spec:        RasterSpec(epsg=32613, bounds=(499950, 4489410, 611460, 4600...
    crs:         epsg:32613
    transform:   | 30.00, 0.00, 499950.00|\n| 0.00,-30.00, 4600860.00|\n| 0.0...
    resolution:  30

In [9]:
%timeit da.compute()

KeyboardInterrupt: 